In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import os
import ast

In [2]:
DATA_PATH = "D:\AnhTuan\BaiTap\MLDatasets_DS423\DoAn\e_learning_recommender\ml\data\processed\cleaned_courses.csv"
MODEL_DIR = "D:\AnhTuan\BaiTap\MLDatasets_DS423\DoAn\e_learning_recommender\ml\models"
TFIDF_VECTORIZER_PATH = os.path.join(MODEL_DIR, "tfidf_vectorizer.joblib")
COSINE_SIM_MATRIX_PATH = os.path.join(MODEL_DIR, "cosine_sim_matrix.joblib")
COURSE_DATA_PATH = os.path.join(MODEL_DIR, "course_data_for_recommendation.csv")

In [3]:
def train_and_save_model():
    """
    Huấn luyện mô hình Content-Based và lưu các thành phần cần thiết.
    """
    print("--- Bắt đầu quá trình huấn luyện mô hình gợi ý ---")

    # 1. Tạo thư mục models nếu chưa tồn tại
    os.makedirs(MODEL_DIR, exist_ok=True)

    # 2. Đọc dữ liệu
    print(f"Đọc dữ liệu từ {DATA_PATH}...")
    try:
        df = pd.read_csv(DATA_PATH)
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file dữ liệu tại '{DATA_PATH}'.")
        print("Vui lòng chạy script data_preprocessing.py trước.")
        return

    # 3. Tiền xử lý cột skills để đưa về dạng chuỗi
    print("Tiền xử lý cột 'processed_skills'...")
    # Chuyển đổi chuỗi '[skill1, skill2]' thành list python
    df["processed_skills"] = df["processed_skills"].apply(ast.literal_eval)
    # Nối các skills trong list thành một chuỗi duy nhất, phân tách bằng khoảng trắng
    df["skills_as_string"] = df["processed_skills"].apply(lambda x: " ".join(x))

    # 4. Huấn luyện TF-IDF Vectorizer
    print("Huấn luyện TF-IDF Vectorizer...")
    # stop_words='english' để loại bỏ các từ phổ biến vô nghĩa trong tiếng Anh
    tfidf_vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = tfidf_vectorizer.fit_transform(df["skills_as_string"])

    # 5. Tính toán ma trận tương đồng Cosine
    print("Tính toán ma trận tương đồng Cosine...")
    cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    print(f"Kích thước ma trận tương đồng: {cosine_sim_matrix.shape}")

    # 6. Lưu các thành phần đã huấn luyện
    print(f"Lưu TF-IDF Vectorizer vào: {TFIDF_VECTORIZER_PATH}")
    joblib.dump(tfidf_vectorizer, TFIDF_VECTORIZER_PATH)

    print(f"Lưu ma trận tương đồng Cosine vào: {COSINE_SIM_MATRIX_PATH}")
    joblib.dump(cosine_sim_matrix, COSINE_SIM_MATRIX_PATH)

    # 7. Lưu một phiên bản DataFrame gọn nhẹ chỉ chứa thông tin cần cho gợi ý
    # Điều này giúp service gợi ý không cần phải load cả file CSV lớn
    course_data_for_recommendation = df[["id", "course_name"]].copy()
    course_data_for_recommendation.to_csv(COURSE_DATA_PATH, index=False)
    print(f"Lưu dữ liệu khóa học cho gợi ý vào: {COURSE_DATA_PATH}")

    print("\nHuấn luyện và lưu mô hình thành công!")


if __name__ == "__main__":
    train_and_save_model()

--- Bắt đầu quá trình huấn luyện mô hình gợi ý ---
Đọc dữ liệu từ D:\AnhTuan\BaiTap\MLDatasets_DS423\DoAn\e_learning_recommender\ml\data\processed\cleaned_courses.csv...
Tiền xử lý cột 'processed_skills'...
Huấn luyện TF-IDF Vectorizer...
Tính toán ma trận tương đồng Cosine...
Kích thước ma trận tương đồng: (3286, 3286)
Lưu TF-IDF Vectorizer vào: D:\AnhTuan\BaiTap\MLDatasets_DS423\DoAn\e_learning_recommender\ml\models\tfidf_vectorizer.joblib
Lưu ma trận tương đồng Cosine vào: D:\AnhTuan\BaiTap\MLDatasets_DS423\DoAn\e_learning_recommender\ml\models\cosine_sim_matrix.joblib
Lưu dữ liệu khóa học cho gợi ý vào: D:\AnhTuan\BaiTap\MLDatasets_DS423\DoAn\e_learning_recommender\ml\models\course_data_for_recommendation.csv

Huấn luyện và lưu mô hình thành công!
